# 建議先從rag-faiss.ipynb開始跑 環境最複雜 但是弄完 4份都可以跑

必要

miniconda_python(3.10版本):

https://docs.conda.io/projects/miniconda/en/latest/miniconda-other-installer-links.html

裝完打開anaconda prompt:

做一個環境```conda create -n VScode-env-3.10.13 python=3.10.13```

做完啟動```conda activate VScode-env-3.10.13```

安裝faiss插件

```conda install -c pytorch faiss-cpu=1.7.4 mkl=2021 blas=1.0=mkl```

```conda install -c pytorch/label/nightly faiss-cpu```

關掉prompt 重新開啟VScode 右上角kernel 選擇another kernel->python env->VScode-env-3.10.13

插件:

沒裝pandas:```pip install pandas```

沒裝openai_SDK使用:```pip install openai```

沒設定key使用:```setx OPENAI_API_KEY “<yourkey>”```

```echo %OPENAI_API_KEY%```可以看你的KEY

讀embeddings資料

並把dataframe轉為numpy

In [8]:
import pandas as pd
import numpy as np

csv_file = 'data\Qembeddings.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,-0.022805,-0.003135,-0.013681,-0.036727,-0.044497,0.029007,-0.024501,-0.041485,-0.005907,-0.009333,...,-0.001190,-0.021287,0.009650,-0.010143,-0.028197,0.025374,-0.010055,-0.003249,-0.007062,-0.028197
1,-0.016840,-0.011369,-0.003460,-0.017197,-0.024066,0.012614,-0.030361,-0.040702,0.012908,-0.003005,...,0.013699,-0.041800,0.021296,-0.006463,-0.004542,-0.020491,-0.033731,0.010705,0.003454,-0.024437
2,-0.020366,-0.011115,0.016573,-0.021214,-0.017512,-0.002729,-0.011475,-0.030523,0.020134,-0.016496,...,0.023915,-0.022757,0.027797,-0.001901,-0.010009,0.003963,-0.034329,-0.025637,0.005378,-0.002792
3,-0.029621,-0.031300,-0.004523,-0.022191,-0.040698,0.019835,-0.013771,-0.023770,0.014309,-0.008721,...,0.015738,-0.005817,0.005927,0.000405,-0.026338,0.000488,-0.020086,-0.000731,0.001592,-0.020474
4,-0.014236,-0.011760,-0.014080,-0.002142,-0.036923,0.014444,-0.035155,-0.033881,0.009361,-0.001141,...,0.014509,-0.018397,0.017265,-0.009296,-0.004875,-0.010771,-0.011649,0.004901,0.003660,-0.035259


確認內容

In [9]:
print(data.shape[0])
print(type(data))
print(data.dtype)
print(data[0][0])

669
<class 'numpy.ndarray'>
float64
-0.0228053741157054


寫入維度 並把data加入faiss的index

In [6]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

# 問題輸入在此

In [7]:
input='hey flora! Im having some issue with password! can you reset it for me?'

做問題的embedding

In [19]:
import os
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

response = client.embeddings.create(
    input=input,
    model="text-embedding-ada-002"
)
question = np.array(response.data[0].embedding, ndmin=2)
print(question)

[[-0.00663677 -0.00735874 -0.03375349 ...  0.01932745 -0.00485504
  -0.02893156]]


TOP-k的設定並查找問題

In [28]:
k = 3

distances, indices = index.search(question, k)

拿答案

In [32]:
csv_question = 'data\manual.csv'
qdf = pd.read_csv(csv_question)
qarray = qdf.to_numpy()

看答案和距離

In [35]:
answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
print(answer)

[(array([' I forgot my password for my newly created Flora account thus I would like to reset my password.',
       'In the "Log In" section, there is an option "Forgot password?". By clicking, it would guide you through the process of resetting the password.Please try logging in with your old login method. If the problem still persists, we could manually migrate your old account to a new login method by the following process: 1) Sign up for a new account 2) Send us both your old and new account information, including login method, user name, and email account. If you use Apple Login without revealing your email, a screenshot of the Flora Setting page including the above mentioned information is needed. Thank you for your support and understanding! '],
      dtype=object), 0.20552632212638855), (array(["I already made an account for Flora and can't remember my password, but the app won't let me click 'forgot password'. Would you be able to send me a link so I can reset my password?",
 

In [42]:
for i in range(0,5):
    print(answer[i][0][1])


In the "Log In" section, there is an option "Forgot password?". By clicking, it would guide you through the process of resetting the password.Please try logging in with your old login method. If the problem still persists, we could manually migrate your old account to a new login method by the following process: 1) Sign up for a new account 2) Send us both your old and new account information, including login method, user name, and email account. If you use Apple Login without revealing your email, a screenshot of the Flora Setting page including the above mentioned information is needed. Thank you for your support and understanding! 
We can help migrate your account. Please use a new email address to register a new account and complete the verification process. After that, send us the old and new email address for the account, and we will manually migrate for you. Note that we can only keep the data of one account after connecting. In other words, the data in another account will be e